# Urban Water Model Simulation for Fehraltorf

This notebook runs the urban water model simulation for Fehraltorf and visualizes the results.

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import geopandas as gpd
from dynaconf import Dynaconf

import holoviews as hv
from bokeh.io import output_notebook
hv.extension('bokeh')
output_notebook()

import plotly.graph_objects as go
from typing import Dict, List
import json
from IPython.display import display, HTML
import ipywidgets as widgets

from duwcm.read_data import read_data
from duwcm.forcing import read_forcing, distribute_irrigation
from duwcm.water_model import UrbanWaterModel
from duwcm.water_balance import run_water_balance
from duwcm.utils import load_config
from duwcm.scenario_manager import ScenarioManager, Scenario, run_scenario
from duwcm.postprocess import extract_local_results

from duwcm.viz import (
    plot_aggregated_results,
    create_map_base,
    create_dynamic_map,
    create_flows,
    create_reuse_flows
)

## Configuration

Set up the configuration for the simulation.

In [ ]:
config = load_config('.', 'default', 'config.yaml')

print(f"Input directory: {config.input_directory}")
print(f"Output directory: {config.output.directory}")
print(f"Simulation period: {config.simulation.start_year} - {config.simulation.end_year}")

geo_file = Path(config.input_directory) / Path(config.files.geo)
_, _, _, _, _, flow_paths = read_data(config)
background_file = Path(config.geodata_directory) / config.files.background_shapefile

base_map = create_map_base(geo_file, background_file, flow_paths)
base_map.show()

## Scenarios
Configure the scenarios

In [ ]:
# Scenarios
scenarios = ['default', 'dry', 'wet', 'water_saving', 'high_consumption']
scenario_results: Dict[str, Dict] = {}

# Create parameter sliders for each scenario
scenario_params = {
    'dry': {
        'precipitation_factor': widgets.FloatSlider(value=0.5, min=0.0, max=2.0, step=0.1, description='Precip. Factor'),
        'indoor_water_factor': widgets.FloatSlider(value=1.0, min=0.0, max=2.0, step=0.1, description='Indoor Water')
    },
    'wet': {
        'precipitation_factor': widgets.FloatSlider(value=2.0, min=0.0, max=2.0, step=0.1, description='Precip. Factor'),
        'indoor_water_factor': widgets.FloatSlider(value=1.0, min=0.0, max=2.0, step=0.1, description='Indoor Water')
    },
    'water_saving': {
        'precipitation_factor': widgets.FloatSlider(value=1.0, min=0.0, max=2.0, step=0.1, description='Precip. Factor'),
        'indoor_water_factor': widgets.FloatSlider(value=0.5, min=0.0, max=2.0, step=0.1, description='Indoor Water')
    },
    'high_consumption': {
        'precipitation_factor': widgets.FloatSlider(value=1.0, min=0.0, max=2.0, step=0.1, description='Precip. Factor'), 
        'indoor_water_factor': widgets.FloatSlider(value=2.0, min=0.0, max=2.0, step=0.1, description='Indoor Water')
    }
}

# Create tabs for each scenario
scenario_tabs = widgets.Tab()
scenario_boxes = []

for scenario in scenarios[1:]:  # Skip default
    vbox = widgets.VBox([
        widgets.HTML(f"<h3>{scenario} Scenario Parameters</h3>"),
        scenario_params[scenario]['precipitation_factor'],
        scenario_params[scenario]['indoor_water_factor']
    ])
    scenario_boxes.append(vbox)

scenario_tabs.children = scenario_boxes
for i, scenario in enumerate(scenarios[1:]):
    scenario_tabs.set_title(i, scenario)

# Display parameter interface
display(scenario_tabs)

# Modify the scenarios loop to use widget values
def get_scenario_config(scenario: str) -> Dynaconf:
    """Get configuration with updated parameters from widgets"""
    scenario_config = load_config('.', 'default', 'scenarios.yaml')
    
    if scenario != 'default' and scenario in scenario_params:
        # Update parameters from widgets
        scenario_config.scenarios[scenario]['precipitation_factor'] = scenario_params[scenario]['precipitation_factor'].value
        scenario_config.scenarios[scenario]['indoor_water_factor'] = scenario_params[scenario]['indoor_water_factor'].value
    
    return scenario_config

## Run Simulation

Execute the urban water model simulation.

In [ ]:
# Read base data
model_params, reuse_settings, demand_data, soil_data, et_data, flow_paths = read_data(config)
forcing_data = read_forcing(config)

model_data = {
    'flow_paths': flow_paths, 
    'soil_data': soil_data,
    'et_data': et_data,
    'demand_data': demand_data,
    'reuse_settings': reuse_settings,
    'direction': config.grid.direction
}

# Create scenario manager with config updated from sliders
scenario_config = get_scenario_config(scenario)  
scenario_manager = ScenarioManager.from_config(scenario_config)

# Run all scenarios
scenario_results = scenario_manager.run_scenarios(model_data=model_data,
                                                base_params=model_params,
                                                base_forcing=forcing_data, 
                                                n_jobs=-1)

## Visualize Results

Create plots to visualize the simulation results.

In [ ]:
def plot_scenario_comparison():
    fig = go.Figure()
    
    for scenario, results in scenario_results.items():
        agg = results['aggregated']
        
        # Add evapotranspiration
        et_sum = (agg['evaporation'] + agg['transpiration']).pint.magnitude
        fig.add_trace(go.Scatter(
            x=agg.index,
            y=et_sum,
            name=f'{scenario} - ET',
            line=dict(dash='solid'),
            visible='legendonly' if scenario != 'default' else True
        ))
        
        # Add runoff
        fig.add_trace(go.Scatter(
            x=agg.index,
            y=agg['stormwater'].pint.magnitude,
            name=f'{scenario} - Runoff',
            line=dict(dash='dot'),
            visible='legendonly' if scenario != 'default' else True
        ))

        # Add sewerage
        fig.add_trace(go.Scatter(
            x=agg.index,
            y=agg['sewerage'].pint.magnitude,
            name=f'{scenario} - Discharge', 
            line=dict(dash='dot'),
            visible='legendonly' if scenario != 'default' else True
        ))

    fig.update_layout(
        title="Scenario Comparison",
        xaxis_title="Date",
        yaxis_title="Flow [m³/day]",
        height=600,
        hovermode='x unified'
    )
    return fig

plot_scenario_comparison().show()

## Summarize Results

Create plots to visualize the simulation results.

In [ ]:
def print_scenario_summary():
    summary = {}
    for scenario, results in scenario_results.items():
        agg = results['aggregated']
        
        summary[scenario] = {
            'Total ET': (agg['evaporation'] + agg['transpiration']).pint.magnitude.sum(),
            'Total Runoff': agg['stormwater'].pint.magnitude.sum(),
            'Total Baseflow': agg['baseflow'].pint.magnitude.sum(),
            'Total Imported Water': agg['imported_water'].pint.magnitude.sum()
        }
    
    df = pd.DataFrame(summary).round(2)
    df['units'] = {
        'Total ET': f"{agg['evaporation'].pint.units:~P}",
        'Total Runoff': f"{agg['stormwater'].pint.units:~P}",
        'Total Baseflow': f"{agg['baseflow'].pint.units:~P}",
        'Total Imported Water': f"{agg['imported_water'].pint.units:~P}"
    }
    return df

display(HTML(print_scenario_summary().to_html()))